In [1]:
import math
import numpy as np
import cv2
import tensorflow as tf
import tensorflow.experimental.numpy as tnp
from PIL import Image
from os import listdir
from datetime import timedelta
import matplotlib.pyplot as plt
from timeit import default_timer as timer

2023-06-22 01:19:51.427284: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.2
2023-06-22 01:20:08.032982: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.2


In [2]:
# Verificar los dispositivos disponibles
devices = tf.config.list_physical_devices('GPU')
if len(devices) > 0:
    print("TensorFlow está utilizando la GPU:")
    for device in devices:
        print(device)
else:
    print("TensorFlow no está utilizando la GPU. Asegúrate de que los controladores de GPU estén instalados correctamente.")


TensorFlow está utilizando la GPU:
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2023-06-21 23:26:58.480778: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-06-21 23:26:58.558795: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:908] ARM64 does not support NUMA - returning NUMA node zero
2023-06-21 23:26:58.559089: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:00.0 name: NVIDIA Tegra X1 computeCapability: 5.3
coreClock: 0.9216GHz coreCount: 1 deviceMemorySize: 1.93GiB deviceMemoryBandwidth: 194.55MiB/s
2023-06-21 23:26:58.559410: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.2
2023-06-21 23:26:58.819330: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-06-21 23:26:58.819652: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10

In [2]:
def filtcosenoF(par,fi,co):

    # Coordenadas
    [xfc, yfc] = np. meshgrid(np.linspace(1-co/2, co/2, co), np.linspace(fi/2, 1-fi/2, fi))

    # Normalizar cooredenadas en intervalo (-pi, pi) y crear filtros en
    # dirección horizontal y vertical
    fc1 = np.cos(xfc*(math.pi/par)*(1/np.max(xfc)))**2
    fc2 = np.cos(yfc*(math.pi/par)*(1/np.max(yfc)))**2

    # Intersectar ambas direcciones
    fc = (fc1 > 0)*fc1*(fc2 > 0)*fc2

    # Re-escalar de 0 a 1
    return fc/np.max(fc)

In [6]:
# Obtener información sobre el dispositivo de una operación o tensor
device = c.device
print("Dispositivo de ejecución:", device)

Dispositivo de ejecución: /job:localhost/replica:0/task:0/device:GPU:0


In [7]:
import tensorflow as tf
import pycuda.driver as cuda

# Obtener el contexto de la GPU
gpu_device_name = '/job:localhost/replica:0/task:0/device:GPU:0'
gpu_device = tf.config.experimental.get_device_details(gpu_device_name)

# Obtener el identificador de la GPU
gpu_id = int(gpu_device_name.split(':')[-1])

# Inicializar el driver de CUDA
cuda.init()

# Obtener el contexto de la GPU
cuda_device = cuda.Device(gpu_id)

# Obtener las especificaciones de la GPU
name = cuda_device.name()
compute_capability = cuda_device.compute_capability()
total_memory = cuda_device.total_memory()

print("Especificaciones de la GPU:")
print("Nombre: ", name)
print("Capacidad de cómputo: ", compute_capability)
print("Memoria total: ", total_memory/(1024**2), "MB")


ModuleNotFoundError: No module named 'pycuda'

In [4]:
def prepairholoF(CH_m,dx,L):

    [row, col] = CH_m.shape
    Wx = dx * col
    Wy = dx * row

    # Matriz coordinates
    [X, Y] = np.meshgrid(range(col), range(row))

    # Hologram origin coordinates
    xo = -Wx / 2
    yo = -Wy / 2

    # Prepared Hologram, coordinates origin
    xop = xo * L / math.sqrt(L ** 2 + xo ** 2)
    yop = yo * L / math.sqrt(L ** 2 + yo ** 2)

    Xp = dx*(X-col/2)*L/np.sqrt(L**2+(dx**2)*(X-col/2)**2+(dx**2)*(Y-row/2)**2)
    Yp = dx*(Y-row/2)*L/np.sqrt(L**2+(dx**2)*(X-col/2)**2+(dx**2)*(Y-row/2)**2)
    # New coordinates measured in units of the -2*xop/(row) pixel size
    Xcoord=(Xp-xop)/(-2*xop/col)
    Ycoord=(Yp-yop)/(-2*xop/row)
    
    # Find lowest integer
    iXcoord = np.floor(Xcoord).astype(np.uint16)
    iYcoord = np.floor(Ycoord).astype(np.uint16)
    
    # Calculate the fractioning for interpolation
    x1frac = (iXcoord + 1.0)-Xcoord                #upper value to integer
    x2frac = 1.0-x1frac                            #lower value to integer
    y1frac = (iYcoord + 1.0)-Ycoord
    y2frac = 1.0-y1frac
    
    x1y1 = x1frac*y1frac                          #Corresponding pixel areas for each direction
    x1y2 = x1frac*y2frac
    x2y1 = x2frac*y1frac
    x2y2 = x2frac*y2frac
    # En éstas matrices estèn los factores de interpolación para la proyección del holograma en un sistema esférico
    # estos facotores estan mapeados con las coordenandas en iCoord, buscar la manera de mapearlos a las coordenadas
    # del holograma. (interpolación bilineal?)
    
    # Pre-allocate the prepared hologram
    CHp_m = np.zeros((row, col))
    
    # Prepare hologram (preparation1 - every pixel remapping)
    for it in range(row-1):
        for jt in range(col-1):
            CHp_m[iYcoord[it, jt], iXcoord[it, jt]] = CHp_m[iYcoord[it, jt], iXcoord[it, jt]] + \
                                                      x1y1[it, jt]*CH_m[it, jt]
            CHp_m[iYcoord[it, jt], iXcoord[it, jt]+1] = CHp_m[iYcoord[it, jt], iXcoord[it, jt]+1] + \
                                                        x2y1[it, jt]*CH_m[it, jt]
            CHp_m[iYcoord[it, jt]+1, iXcoord[it, jt]] = CHp_m[iYcoord[it, jt]+1, iXcoord[it, jt]] + \
                                                        x1y2[it, jt]*CH_m[it, jt]
            CHp_m[iYcoord[it, jt]+1, iXcoord[it, jt]+1] = CHp_m[iYcoord[it, jt]+1, iXcoord[it, jt]+1] + \
                                                          x2y2[it, jt]*CH_m[it, jt]
    return CHp_m

In [5]:
def prepairholoF_remap(CH_m,dx,L):

    [row, col] = CH_m.shape
    Wx = dx * col
    Wy = dx * row

    # Matriz coordinates
    [X, Y] = np.meshgrid(range(col), range(row))
    X = X - col/2
    Y = Y - row/2

    r = np.sqrt(X**2 + Y**2)
    rW = np.sqrt((Wx/2)**2 + (Wy/2)**2)
    auxL = np.max(r)*L/np.max(rW)
    theta = np.arctan2(Y, X)
    R = auxL*np.sin(r/auxL)

    Xcoord = R*np.cos(theta)
    Ycoord = R*np.sin(theta)
    Xcoord = (Xcoord+col/2).astype(np.float32)
    Ycoord = (Ycoord+row/2).astype(np.float32)
    CHp_m = cv2.remap(CH_m, Xcoord, Ycoord, cv2.INTER_AREA, borderMode=cv2.BORDER_CONSTANT, borderValue=0)

    return CHp_m

In [6]:
def kreuzer3F(CHp_m, z, L, lamb, deltaX, FC):

    [row, col] = CHp_m.shape
    Wx = dx * col
    Wy = dx * row

    # Hologram origin coordinates
    xo = -Wx / 2
    yo = -Wy / 2

    # Prepared Hologram, coordinates origin
    xop = xo * L / math.sqrt(L ** 2 + xo ** 2)
    yop = yo * L / math.sqrt(L ** 2 + yo ** 2)

    #np to TNP (TensorNumPy)
    CHp_m = tnp.asarray(CHp_m)
    z = tnp.asarray(z)
    L = tnp.asarray(L)
    xop = tnp.asarray(xop)
    yop = tnp.asarray(yop)
    deltaX = tnp.asarray(deltaX)

    # Square pixels:
    deltaY = deltaX

    # Matrix size
    [row, col] = CHp_m.shape

    # Parameters
    k = tnp.asarray(2*math.pi/lamb)

    # Matriz coordinates
    [X, Y] = tnp.meshgrid(range(col), range(row))
    X = X.astype(tnp.float64)
    Y = Y.astype(tnp.float64)
    # Pixel size for the prepared hologram
    deltaxp = xop / (-col/2)
    deltayp = yop / (-row/2)

    # Coordinates origin for the reconstruction plane
    Yo = -deltaX * (row/2)
    Xo = -deltaX * (col/2)

    # Multiply prepared hologram with propagation phase
    Rp = tnp.sqrt((L**2)-(deltaxp*X+xop)**2-(deltayp*Y+yop)**2)
    r = tnp.sqrt((deltaX**2)*((X-col/2)**2+(Y-row/2)**2)+z**2)
    CHp_m = CHp_m*((L/Rp)**4)*np.exp(-0.5j*k*(r**2-2*z*L)*Rp/(L**2))

    #.* exp(0.125 * 1i * k * ((L ^ 2)./ Rp).* ((r.^ 2 - 2 * z * L).* (Rp./ (L ^ 2)).^ 2).^ 2);...
    #.* exp(-(3 / 48) * 1i * k * ((L ^ 2)./ Rp).* ((r.^ 2 - 2 * z * L).* (Rp./ (L ^ 2)).^ 2).^ 3)...
    #.* exp((15 / 384) * 1i * k * ((L ^ 2)./ Rp).* ((r.^ 2 - 2 * z * L).* (Rp./ (L ^ 2)).^ 2).^ 4);

    # Padding constant value
    pad = int(col/2)

    # Padding on the cosine rowlter
    FC = np.pad(FC, pad)

    # Convolution operation
    # First transform
    T1 = CHp_m*tnp.exp((1j*k/(2*L))*(2*Xo*X*deltaxp+2*Yo*Y*deltayp+X**2*deltaxp*deltaX+Y**2*deltayp*deltaY))
    T1 = np.pad(T1, pad)
    T1 = tf.signal.fftshift(tf.signal.fft2d(tf.signal.fftshift(T1*FC)))
    tf.keras.backend.clear_session()

    # Second transform
    T2 = tnp.exp(-1j*(k/(2 * L)) * ((X-col/2)**2 * deltaxp * deltaX + (Y-row/2)**2 * deltayp * deltaY))
    T2 = np.pad(T2, pad)
    T2 = tf.signal.fftshift(tf.signal.fft2d(tf.signal.fftshift(T2*FC)))
    tf.keras.backend.clear_session()

    # Third transform
    K = tf.signal.ifftshift(tf.signal.ifft2d(tf.signal.ifftshift(T2*T1)))
    tf.keras.backend.clear_session()
    K = K[pad:(pad+row), pad:(pad+col)]

    # Multiply by aditional terms after the propagation
    # K = K * deltaxp * deltayp * (exp(sqrt(-1) * (k / L) * ((Xo + X * deltaX) * xop + (Yo + Y * deltaY) * yop)))...
    # .*exp(sqrt(-1) * (0.5 * k/L) * ((X - 0 * col/2)**2 * deltaxp * deltaX + (Y - 0 * row / 2)**2 *deltayp*deltaY))

    return K

In [7]:
filename = r'/home/jetson/Downloads/H05 (3).png'
holo = np.array(Image.open(filename))

In [13]:
holo= cv2.resize(holo,(128,128),interpolation=cv2.INTER_LINEAR)

In [8]:
[fi, co,] = holo.shape
holo = holo[:, int((co-fi)/2):co-int((co-fi)/2)]
[fi, co] = holo.shape

In [9]:
from tensorflow.keras import backend as K
import gc

def limpiar():
    K.clear_session()  # limpiar la memoria
    gc.collect()

In [10]:
limpiar()

In [11]:
holoContrast = holo  

In [12]:
fold_files = listdir('.')
stored_filt = list(filter(lambda x: 'filtcos.npy' in x, fold_files))

In [13]:
if len(list(filter(lambda x: (str(co)+'-'+str(fi)) in x, fold_files))) == 0:
    fc = filtcosenoF(100, fi, co)
    np.save(str(co)+'-'+str(fi)+'_filtcos.npy', fc)
    print('Cosine filter saved')
else:
    fc = np.load(str(co)+'-'+str(fi)+'_filtcos.npy')
    print('Cosine filter loaded')

Cosine filter loaded


In [14]:
limpiar()

In [15]:
z = 5e-3
L = 8e-3
lamb = 532e-9
dx = 1.55e-6 #tamaño del pixel

# LET'S FIND THE BEST FOCUSED RECONSTRUCTION
# DLHM reconstruction
# pixel size at reconstruction plane
deltaX = z*dx/L  # in micrometers

In [16]:
pre_holo = prepairholoF_remap(holoContrast, dx, L) # Faster (0.11 Seconds)

In [17]:
reconstruction = kreuzer3F(pre_holo, z, L, lamb, deltaX, fc)

2023-06-22 01:24:37.270758: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-06-22 01:24:37.331571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:908] ARM64 does not support NUMA - returning NUMA node zero
2023-06-22 01:24:37.350936: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:00.0 name: NVIDIA Tegra X1 computeCapability: 5.3
coreClock: 0.9216GHz coreCount: 1 deviceMemorySize: 1.93GiB deviceMemoryBandwidth: 194.55MiB/s
2023-06-22 01:24:37.351177: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.2
2023-06-22 01:24:37.551019: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-06-22 01:24:37.551451: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10

ResourceExhaustedError: OOM when allocating tensor with shape[1040,1040] and type double on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Pow]